# RePath Model Release Workflow\n\nThis notebook walks through building and validating a semver model release bundle, then publishing a GitHub release.

## 1. Prerequisites\n- Run this notebook from the `repath-model` repo root.\n- `gh` CLI authenticated (`gh auth status`).\n- `release-notes.md` updated for the target release.

In [ ]:
import os\nimport subprocess\nfrom pathlib import Path\n\nROOT = Path.cwd()\nVERSION = "1.0.0"  # set target version\nTAG = f"v{VERSION}"\nSOURCE_RUN_ID = "20260217-001"  # set candidate/run id\nMODEL_PATH = Path("../repath-mobile/assets/models/yolo-repath.tflite").resolve()\nLABELS_PATH = Path("../repath-mobile/assets/models/yolo-repath.labels.json").resolve()\nNOTES_PATH = Path("release-notes.md").resolve()\n\nprint({\n    "root": str(ROOT),\n    "version": VERSION,\n    "model": str(MODEL_PATH),\n    "labels": str(LABELS_PATH),\n    "release_notes": str(NOTES_PATH),\n})

In [ ]:
# Build release bundle\nsubprocess.run([\n    "python3", "scripts/build_release.py",\n    "--version", VERSION,\n    "--model", str(MODEL_PATH),\n    "--labels", str(LABELS_PATH),\n    "--source-run-id", SOURCE_RUN_ID,\n    "--notes-file", str(NOTES_PATH),\n], check=True)

In [ ]:
# Verify generated release manifest checksums\nmanifest = ROOT / f"dist/releases/{TAG}/release-manifest-{TAG}.json"\nsubprocess.run([\n    "python3", "scripts/verify_release.py",\n    "--manifest", str(manifest),\n], check=True)

In [ ]:
# Inspect bundle files\nrelease_dir = ROOT / f"dist/releases/{TAG}"\nfor p in sorted(release_dir.glob("*")):\n    print(p.name, p.stat().st_size)

## 2. Publish to GitHub Releases\nUncomment and run these cells when ready to publish.

In [ ]:
# subprocess.run(["git", "tag", TAG], check=True)\n# subprocess.run(["git", "push", "origin", TAG], check=True)

In [ ]:
# subprocess.run([\n#     "gh", "release", "create", TAG,\n#     f"dist/releases/{TAG}/yolo-repath-{TAG}.tflite",\n#     f"dist/releases/{TAG}/yolo-repath-{TAG}.labels.json",\n#     f"dist/releases/{TAG}/release-manifest-{TAG}.json",\n#     f"dist/releases/{TAG}/SHA256SUMS",\n#     "--repo", "krispeterson/repath-model",\n#     "--title", f"RePath Model {TAG}",\n#     "--notes-file", str(NOTES_PATH),\n# ], check=True)